This notebook shows that the continuous and discrete adjoint methods give the same result if no stabilisation is applied.

In [1]:
from firedrake import *
from firedrake_adjoint import *

In [2]:
import matplotlib.pyplot as plt
import scipy.interpolate as si

In [3]:
from adapt_utils.case_studies.tohoku.options.options import TohokuInversionOptions
from adapt_utils.misc import gaussian, ellipse

Specify 'optimum' and initial guess

In [4]:
m_opt = 5.0
m_prior = 10.0

Create `Options` object and setup gauges.

In [5]:
level = 3
op = TohokuInversionOptions(level=level)
gauges = list(op.gauges.keys())
for gauge in gauges:
    if gauge[:2] not in ('P0', '80'):
        op.gauges.pop(gauge)
gauges = list(op.gauges.keys())

/home/joe/software/firedrake-pragmatic/src/firedrake/firedrake/parloops.py:130: LoopyWarning: 'lang_version' was not passed to make_function(). To avoid this warning, pass lang_version=(2018, 2) in this invocation. (Or say 'from loopy.version import LOOPY_USE_LANGUAGE_VERSION_2018_2' in the global scope of the calling frame.)
  knl = loopy.make_function(kernel_domains, instructions, kargs, seq_dependencies=True,


Setup timestepping parameters

In [6]:
num_timesteps = 30*2**level
dt = Constant(op.dt)
theta = Constant(0.5)

Create function spaces

In [7]:
mesh = op.default_mesh
P2 = VectorFunctionSpace(mesh, "CG", 1)
P1 = FunctionSpace(mesh, "CG", 1)
V = P2*P1
R = FunctionSpace(mesh, "R", 0)

Create `Function`s

In [8]:
u, eta = TrialFunctions(V)
test_u, test_eta = TestFunctions(V)
m = Function(R).assign(m_prior)
c = Control(m)
q0 = Function(V)
u0, eta0 = q0.split()
eta0.interpolate(gaussian([(0.7e+06, 4.2e+06, 48e+03, 96e+03)], mesh, rotation=pi/12))
q_ = Function(V)
u_, eta_ = q_.split();

Set physical parameters

In [9]:
b = Function(P1).assign(op.set_bathymetry(P1))
g = Constant(9.81)

Setup forward variational problem

In [10]:
a = inner(u, test_u)*dx + eta*test_eta*dx
a += theta*dt*g*inner(grad(eta), test_u)*dx - theta*dt*b*inner(u, grad(test_eta))*dx
L = inner(u_, test_u)*dx + eta_*test_eta*dx
L += -(1-theta)*dt*g*inner(grad(eta_), test_u)*dx + (1-theta)*dt*b*inner(u_, grad(test_eta))*dx
q = Function(V)
u, eta = q.split()
bc = DirichletBC(V.sub(1), 0, 100)
problem = LinearVariationalProblem(a, L, q, bcs=bc)

Setup forward variational solver

In [11]:
sp = {
    "snes_type": "ksponly",
    "ksp_type": "gmres",
    "pc_type": "fieldsplit",
    "pc_fieldsplit_type": "multiplicative",
}
solver = LinearVariationalSolver(problem, solver_parameters=sp)

Setup QoI

In [12]:
P0 = FunctionSpace(mesh, "DG", 0)
J_form = 0
radius = 20e+03*0.5**level
for gauge in gauges:
    op.gauges[gauge]["data"] = []
    k = Function(P0*P0)
    ku, keta = k.split()
    keta.interpolate(ellipse([op.gauges[gauge]["coords"] + (radius,)], mesh))
    keta.assign(keta/assemble(keta*dx))
    op.gauges[gauge]["indicator"] = keta
    op.gauges[gauge]["obs"] = Function(R)
    op.gauges[gauge]["obs_old"] = Function(R)
    J_form += theta*0.5*dt*keta*(eta - op.gauges[gauge]["obs"])**2*dx
    J_form += (1-theta)*0.5*dt*keta*(eta_ - op.gauges[gauge]["obs_old"])**2*dx

Solve forward to generate 'data'

In [13]:
u_.assign(u0)
eta_.project(m_opt*eta0)
for gauge in gauges:
    op.gauges[gauge]["data"].append(float(eta_.at(op.gauges[gauge]["coords"])))
for i in range(num_timesteps):
    print(f"t = {i*op.dt/60:5.1f} min  ||u|| = {norm(u):.4e}  ||eta|| = {norm(eta):.4e}")
    solver.solve()
    q_.assign(q)
    for gauge in gauges:
        op.gauges[gauge]["data"].append(float(eta.at(op.gauges[gauge]["coords"])))
print(f"t = {(i+1)*op.dt/60:5.1f} min  ||u|| = {norm(u):.4e}  ||eta|| = {norm(eta):.4e}")

t =   0.0 min  ||u|| = 0.0000e+00  ||eta|| = 0.0000e+00
t =   0.1 min  ||u|| = 7.2774e+02  ||eta|| = 4.2494e+05
t =   0.2 min  ||u|| = 1.4536e+03  ||eta|| = 4.2420e+05
t =   0.4 min  ||u|| = 2.1756e+03  ||eta|| = 4.2296e+05
t =   0.5 min  ||u|| = 2.8920e+03  ||eta|| = 4.2125e+05
t =   0.6 min  ||u|| = 3.6010e+03  ||eta|| = 4.1908e+05
t =   0.8 min  ||u|| = 4.3009e+03  ||eta|| = 4.1648e+05
t =   0.9 min  ||u|| = 4.9902e+03  ||eta|| = 4.1346e+05
t =   1.0 min  ||u|| = 5.6674e+03  ||eta|| = 4.1006e+05
t =   1.1 min  ||u|| = 6.3311e+03  ||eta|| = 4.0631e+05
t =   1.2 min  ||u|| = 6.9803e+03  ||eta|| = 4.0224e+05
t =   1.4 min  ||u|| = 7.6137e+03  ||eta|| = 3.9789e+05
t =   1.5 min  ||u|| = 8.2306e+03  ||eta|| = 3.9329e+05
t =   1.6 min  ||u|| = 8.8300e+03  ||eta|| = 3.8848e+05
t =   1.8 min  ||u|| = 9.4115e+03  ||eta|| = 3.8350e+05
t =   1.9 min  ||u|| = 9.9745e+03  ||eta|| = 3.7837e+05
t =   2.0 min  ||u|| = 1.0519e+04  ||eta|| = 3.7315e+05
t =   2.1 min  ||u|| = 1.1044e+04  ||eta|| = 3.6

t =  18.4 min  ||u|| = 3.0506e+04  ||eta|| = 2.9575e+05
t =  18.5 min  ||u|| = 3.0693e+04  ||eta|| = 2.9585e+05
t =  18.6 min  ||u|| = 3.0881e+04  ||eta|| = 2.9594e+05
t =  18.8 min  ||u|| = 3.1071e+04  ||eta|| = 2.9604e+05
t =  18.9 min  ||u|| = 3.1262e+04  ||eta|| = 2.9614e+05
t =  19.0 min  ||u|| = 3.1454e+04  ||eta|| = 2.9624e+05
t =  19.1 min  ||u|| = 3.1649e+04  ||eta|| = 2.9634e+05
t =  19.2 min  ||u|| = 3.1845e+04  ||eta|| = 2.9645e+05
t =  19.4 min  ||u|| = 3.2043e+04  ||eta|| = 2.9655e+05
t =  19.5 min  ||u|| = 3.2244e+04  ||eta|| = 2.9665e+05
t =  19.6 min  ||u|| = 3.2446e+04  ||eta|| = 2.9676e+05
t =  19.8 min  ||u|| = 3.2650e+04  ||eta|| = 2.9686e+05
t =  19.9 min  ||u|| = 3.2857e+04  ||eta|| = 2.9696e+05
t =  20.0 min  ||u|| = 3.3066e+04  ||eta|| = 2.9705e+05
t =  20.1 min  ||u|| = 3.3277e+04  ||eta|| = 2.9715e+05
t =  20.2 min  ||u|| = 3.3490e+04  ||eta|| = 2.9724e+05
t =  20.4 min  ||u|| = 3.3705e+04  ||eta|| = 2.9732e+05
t =  20.5 min  ||u|| = 3.3923e+04  ||eta|| = 2.9

Solve forward to annotate to tape

In [14]:
J = 0
u_.assign(u0)
eta_.project(m*eta0)
solutions = [q_.copy(deepcopy=True)]
for i in range(num_timesteps):
    print(f"t = {i*op.dt/60:5.1f} min  ||u|| = {norm(u):.4e}  ||eta|| = {norm(eta):.4e}")
    solver.solve()
    solutions.append(q.copy(deepcopy=True))
    for gauge in gauges:
        op.gauges[gauge]["obs"].assign(op.gauges[gauge]["data"][i+1])
        op.gauges[gauge]["obs_old"].assign(op.gauges[gauge]["data"][i])
    J = J + assemble(J_form)
    q_.assign(q)
J10 = float(J)
print(f"t = {(i+1)*op.dt/60:5.1f} min  ||u|| = {norm(u):.4e}  ||eta|| = {norm(eta):.4e}")
stop_annotating();

t =   0.0 min  ||u|| = 4.9437e+04  ||eta|| = 3.0147e+05
t =   0.1 min  ||u|| = 1.4555e+03  ||eta|| = 8.4988e+05
t =   0.2 min  ||u|| = 2.9071e+03  ||eta|| = 8.4839e+05
t =   0.4 min  ||u|| = 4.3511e+03  ||eta|| = 8.4592e+05
t =   0.5 min  ||u|| = 5.7839e+03  ||eta|| = 8.4250e+05
t =   0.6 min  ||u|| = 7.2019e+03  ||eta|| = 8.3816e+05
t =   0.8 min  ||u|| = 8.6018e+03  ||eta|| = 8.3295e+05
t =   0.9 min  ||u|| = 9.9804e+03  ||eta|| = 8.2692e+05
t =   1.0 min  ||u|| = 1.1335e+04  ||eta|| = 8.2012e+05
t =   1.1 min  ||u|| = 1.2662e+04  ||eta|| = 8.1262e+05
t =   1.2 min  ||u|| = 1.3961e+04  ||eta|| = 8.0448e+05
t =   1.4 min  ||u|| = 1.5227e+04  ||eta|| = 7.9578e+05
t =   1.5 min  ||u|| = 1.6461e+04  ||eta|| = 7.8658e+05
t =   1.6 min  ||u|| = 1.7660e+04  ||eta|| = 7.7696e+05
t =   1.8 min  ||u|| = 1.8823e+04  ||eta|| = 7.6699e+05
t =   1.9 min  ||u|| = 1.9949e+04  ||eta|| = 7.5674e+05
t =   2.0 min  ||u|| = 2.1037e+04  ||eta|| = 7.4629e+05
t =   2.1 min  ||u|| = 2.2087e+04  ||eta|| = 7.3

OSError: [Errno 12] Cannot allocate memory

Create `ReducedFunctional` and compute gradient using discrete adjoint

In [ ]:
Jhat = ReducedFunctional(J, c)

Sample control space at two more points

In [ ]:
J2 = Jhat(m.assign(2.0))
J5 = Jhat(m.assign(5.0))

In [ ]:
l = si.lagrange([2, 5, 10], [J2, J5, J10])
dl = l.deriv()
l_min = -dl.coefficients[1]/dl.coefficients[0]
print("Minimiser of quadratic: {:.4f}".format(l_min))

Plot the parameter space

In [ ]:
fig, axes = plt.subplots(figsize=(8, 8))
xx = np.linspace(2, 10, 100)
axes.plot(xx, l(xx), ':', color='C0')
axes.plot(l_min, l(l_min), '*', markersize=14, color='C0', label=r"$m^\star={:.4f}$".format(l_min))

axes.set_xlabel("Control parameter")
axes.set_ylabel("Quantity of Interest")
axes.grid(True)
axes.legend();

Minimisers for $\theta=\frac12$: 4.0913, 4.7927, 4.9221.
Minimisers for $\theta=1$: 4.5361, 4.8471, 4.9342